# Information Retrieval Practice

Elasticsearch is an open-source distributed search server built on top of Apache Lucene. It’s a great tool that allows to quickly build applications with full-text search capabilities. The core implementation is in Java, but it provides a nice REST interface which allows to interact with Elasticsearch from any programming language.


## Install Elastic Search

To install elastic search download your the package for your platform from Get Elasticsearch
 in https://www.elastic.co/es/start


![](https://github.com/acastellanos-ie/natural_language_processing/blob/master/ir_practice/download.png?raw=1)

Once downloaded, unzip the tar.gz file and run `bin/elasticsearch` (or `bin\elasticsearch.bat` on Windows). This will launch the ElasticSearch Server. Once the server is running, by default it's accessible at [localhost:9200](http://localhost:9200).

## Querying Elastic Search via Python

To make queries to ElasticSearch you can directly query the server endpoint via REST. However, we can make it easier via the the `elasticsearch-py` Python library. This library provides a wrapper for the REST endpoint that will allow us to query the server form Python

In case you have not yet installed the libraries, you can execute the following code

In [1]:
! pip install elasticsearch-dsl
! pip install elasticsearch

In [2]:
import warnings
warnings.filterwarnings('ignore')

from elasticsearch import Elasticsearch
from elasticsearch_dsl import Search, Q, Index
from pprint import pprint

# Exercise 0: Indexing and Searching Demo for ElasticSearch

Now it's time to run some demo program. In this practice, we will create inverted index of sample documents (indexing) and then use Elasticsearch query grammar to search documents (searching).

### Useful functions

Functions to facilitate the reading of the dataset

In [3]:
import os, io
from collections import namedtuple

# A document class with following attributes
# filename: document filename
# text: body of documment
# path: path of document
Doc = namedtuple('Doc', 'filename path text')

def read_doc(doc_path, encoding):
    '''
        reads a document from path
        input:
            - doc_path : path of document
            - encoding: encoding
        output: =>
            - doc: instance of Doc namedtuple
    '''
    filename = doc_path.split('/')[-1]
    fp = io.open(doc_path, 'r', encoding = encoding)
    text = fp.read().strip()
    fp.close()
    return Doc(filename = filename, text = text, path = doc_path)

def read_dataset(path, encoding = "ISO-8859-1"):
    '''
        reads multiple documents from path
        input:
            - doc_path : path of document
            - encoding: encoding
        output: =>
            - docs: instances of Doc namedtuple returned as generator
    '''
    for root, dirs, files in os.walk(path):
        for doc_path in files:
            yield read_doc(root + '/' + doc_path, encoding)

Setting up the connector

To index the documents, we first need to make a connection to **Elasticsearch**.

In [4]:
es_conn = Elasticsearch(
    'https://localhost:9020',
)

es_conn

<Elasticsearch([{'host': 'localhost', 'port': 9020, 'use_ssl': True}])>

##  Indexing

We will try to index the sample documents in `./sample_documents`. To index the documents, we first need to make a connection to **Elasticsearch**. 

Before we index the documents, we first need to define the **configuration of elasticsearch**. During this process, you can define basic configuration of indexer such as tokenizer, stemmer, lemmatizer, and also define which search algorithm elasticsearch will use for search.

Below code shows a simple configuration settings for this demo.
The configuration tells elasticsearch that our document `doc` will have three fields `filename`, `path`, and `text`, and we will use `text` field for search. `my_analyzer` will be used to parse the `text` field, and `my_analyzer` will also be used as a search analyzer, which will parse search queries later on. `index:False` in `filename` and `path` fields tell elasticsearch that we will not index these two fields, therefore, we cannot search these two fields with queries. 

The detailed documentation of analyzer can be found [here](https://www.elastic.co/guide/en/elasticsearch/reference/current/analysis.html).

`"similarity": "boolean"` in `text` field will let elasticsearch know that we will use a boolean search algorithm to search `text` field. The detailed documentation of search algorithms can be found [here](https://www.elastic.co/guide/en/elasticsearch/reference/current/search.html)  and [here](https://www.elastic.co/guide/en/elasticsearch/guide/master/search-in-depth.html). 


In [5]:
# configuration for indexing
settings = {
  "mappings": {
      "properties": {
        "filename": {
          "type": "keyword",
          "index": False,
        },
        "path": {
          "type": "keyword",
          "index": False,
        },
        "text": {
          "type": "text",
          "similarity": "boolean",
          "analyzer": "my_analyzer",
          "search_analyzer": "my_analyzer"
        }
      }
  },    
  "settings": {      
    "analysis": {
      "analyzer": {
        "my_analyzer": {
          "filter": [
            "lowercase","stop"
          ],
          "type": "custom",
          "tokenizer": "whitespace",
          "char_filter": ["my_char_filter"]
        }
      },
      "char_filter": {
        "my_char_filter": {
          "type": "html_strip",
          "escaped_tags": ["b"]
        }
      }
    }
  }
}

Now we will retrieve `sample documents` and indexing them into `INDEX_NAME` index. To that end, the following 2 functions will help you in the creation of the index and the indexing of the documents.


In [6]:
ES_HOSTS = ['http://localhost:9200']
INDEX_NAME = 'sample_index'
DOCS_PATH = 'practice_data/sample_documents'

def create_index(es_conn, index_name, settings):
    '''
        create index structure in elasticsearch server. 
        If index_name exists in the server, it will be removed, and new index will be created.
        input:
            - es_conn: elasticsearch connection object
            - index_name: name of index to create
            - settings: settings and mappings for index to create
        output: =>
            - None
    '''
    if es_conn.indices.exists(index_name):
        es_conn.indices.delete(index = index_name)
        print('index `{}` deleted'.format(index_name))
    es_conn.indices.create(index = index_name, body = settings)
    print('index `{}` created'.format(index_name))            
            
def build_index(es_conn, dataset, index_name, settings, DOC_TYPE='doc'):
    '''
        build index from a collection of documents
        input:
            - es_conn: elasticsearch connection object
            - dataset: iterable, collection of namedtuple Doc objects
            - index_name: name of the index where the documents will be stored
            - DOC_TYPE: type signature of documents
    '''
    # create the index if it doesn't exist
    create_index(es_conn = es_conn, index_name = index_name, settings=settings)
    counter_read, counter_idx_failed = 0, 0 # counters

    # retrive & index documents
    for doc in dataset:
        res = es_conn.index(
            index = index_name,
            id = doc.filename,
            body = doc._asdict())
        counter_read += 1

        if res['result'] != 'created':
            conter_idx_failed += 1
        elif counter_read % 500 == 0:
            print('indexed {} documents'.format(counter_read))

    print('indexed {} docs to index `{}`, failed to index {} docs'.format(
        counter_read,
        index_name,
        counter_idx_failed
    ))
    
    # refresh after indexing
    es_conn.indices.refresh(index=index_name)  

In [ ]:
dataset = read_dataset(DOCS_PATH)
build_index(es_conn, dataset, INDEX_NAME, settings)

We successfully created an inverted index for the sample documents in `./sample/documents`. It's time to search the documents with some queries.

## Searching

### Full-Text Search

The two most important aspects of full-text search are as follows:

##### Relevance

>The ability to rank results by how relevant they are to the given query, whether relevance is calculated using TF/IDF (see [What Is Relevance?](https://www.elastic.co/guide/en/elasticsearch/guide/master/relevance-intro.html)), proximity to a geolocation, fuzzy similarity, or some other algorithm.

##### Analysis

>The process of converting a block of text into distinct, normalized tokens (see [Analysis and Analyzers](https://www.elastic.co/guide/en/elasticsearch/guide/master/analysis-intro.html) in order to (a) create an inverted index and (b) query the inverted index.

#### Term-Based Versus Full-Text

Two types of text query:

##### Term-based

Queries like the term or fuzzy queries are low-level queries that have no analysis phase. They operate on a single term. A term query for the term Foo looks for that exact term in the inverted index and calculates the TF/IDF relevance _score for each document that contains the term.

##### Full-text queries

Queries like the match or query_string queries are high-level queries that understand the mapping of a field:

* If you use them to query a date or integer field, they will treat the query string as a date or integer, respectively.

* If you query an exact value (not_analyzed) string field, they will treat the whole query string as a single term.

* But if you query a full-text (analyzed) field, they will first pass the query string through the appropriate analyzer to produce the list of terms to be queried.

Once the query has assembled a list of terms, it executes the appropriate low-level query for each of these terms, and then combines their results to produce the final relevance score for each document.

#### The match Query

We will perform now different types of queries.

First, a query with a single term

In [ ]:
s = Search(using=es_conn, index="sample_index")
s = s.query("match", text={"query": "obama"})
res = s.execute()

for hit in res:
    print(hit.filename, hit.text[:100], '... - Score:', hit.meta.score)
    print()

#### Multiword Queries

Obviously, we can search on more than one word at a time:

In [ ]:
s = Search(using=es_conn, index="sample_index")
s = s.query("match", text={"query":    "Obama Hillary"})
res = s.execute()

for hit in res:
    print(hit.filename, hit.text[:200], '... - Score:', hit.meta.score)
    print()

doc1.txt Barack Hussein Obama II (born August 4, 1961) is the 44th and current President of the United States, the first African American to hold the office. He served as the junior United States Senator from  ... - Score: 2.0

doc2.txt Michelle LaVaughn Robinson Obama (born January 17, 1964) is the wife of the forty-fourth President of the United States, Barack Obama, and is the first African-American First Lady of the United States ... - Score: 1.0

doc3.txt Joseph Robinette "Joe" Biden, Jr. (born November 20, 1942) is the 47th and current Vice President of the United States. He was a United States Senator from Delaware from January 3, 1973 until his resi ... - Score: 1.0

doc4.txt Hillary Diane Rodham Clinton (born October 26, 1947) is the 67th United States Secretary of State, serving within the administration of President Barack Obama. She was a United States Senate from New  ... - Score: 1.0

doc5.txt John Sidney McCain III (born August 29, 1936) is the senior United States Senat

The important thing is: any document whose title field contains at least one of the specified terms will match the query. The more terms that match, the more relevant the document.

But what happens if I want both terms appearing in the document.

In [ ]:
s = Search(using=es_conn, index="sample_index")
s = s.query("match", text={
    "query":    "Obama Hillary",
    "operator": "and"})
res = s.execute()

for hit in res:
    print(hit.filename, hit.text, '... - Score:', hit.meta.score)
    print()


doc1.txt Barack Hussein Obama II (born August 4, 1961) is the 44th and current President of the United States, the first African American to hold the office. He served as the junior United States Senator from Illinois from January 2005 until he resigned after his election to the presidency in November 2008.

Obama is a graduate of Columbia University and Harvard Law School, where he was the president of the Harvard Law Review. He was a community organizer in Chicago before earning his law degree. He worked as a civil rights attorney in Chicago and also taught constitutional law at the University of Chicago Law School from 1992 to 2004.

Obama served three terms in the Illinois Senate from 1997 to 2004. Following an unsuccessful bid for a seat in the U.S. House of Representatives in 2000, Obama ran for United States Senate in 2004. His victory, from a crowded field, in the March 2004 Democratic primary raised his visibility. His prime-time televised keynote address at the Democratic Nat

And now containing a term but NOT the other.

In [ ]:
# Boolean Query "Obama BUT Hillary"
s = Search(using=es_conn, index="sample_index")
s = s.query("bool", 
            must = [Q('match', text="hillary")],
            must_not = [Q('match', text="obama")]
           )

res = s.execute()

for hit in res:
    print(hit.filename, hit.text[:100], '... - Score:', hit.meta.score)
    print()

doc4.txt Hillary Diane Rodham Clinton (born October 26, 1947) is the 67th United States Secretary of State, s ... - Score: 1.0

